<a href="https://colab.research.google.com/github/maham-gif/AdvisorAgent-using-LangChain-Flan-T5/blob/main/Complete%20DataAnalyst%20Agent%20With%20User%20Interaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ STEP 1: Install Dependencies
!pip install -q openpyxl pandas transformers accelerate sentence-transformers matplotlib

# ✅ STEP 2: Import Libraries
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import io

# ✅ STEP 3: Upload File
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_excel(io.BytesIO(uploaded[filename]))
print("✅ File uploaded and read successfully!\n")
print(df.head(3))

# ✅ STEP 4: User Options
print("🎯 What would you like to do?")
print("""
Available Tasks:
1. clean - Clean dataset (remove duplicates, trim whitespace, drop empty rows)
2. fill_missing - Fill missing values intelligently
3. analyze - Describe data, generate insights
4. generate_rows - Generate new rows based on existing pattern
5. delete_rows - Delete specified number of rows
6. create_chart - Create a chart
7. make_summary - Generate natural language summary of the dataset
8. field_meaning - Understand column names via embeddings
""")

task = input("Enter task: ").strip().lower()

# ✅ STEP 5: Load Hugging Face LLM
model_id = "gpt2"  # Lightweight public model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

# ✅ STEP 6: Optional Embeddings Model (RAG help)
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# ✅ STEP 7: Prompt LLM (Safe Prompt)
prompt = f"""
You are a professional data analyst working on Excel datasets.
This is the top of the dataset:

{df.head(5).to_string(index=False)}

Your task: {task}.
Write clean, safe Python (pandas) code to perform the task.
"""

response = llm(prompt, max_new_tokens=500, temperature=0.7)[0]["generated_text"]
print("\n🧠 Suggested Code by LLM:\n")
print(response)

# ✅ STEP 8: Interactive Actions Per Task
if task == "clean":
    print("⚙️ Cleaning dataset: removing duplicates, empty rows, trimming strings...")
    df = df.drop_duplicates()
    df = df.dropna(how='all')
    df.columns = df.columns.str.strip()
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    print("✅ Dataset cleaned.")

elif task == "fill_missing":
    method = input("Choose fill method (ffill, bfill, mean): ").lower()
    if method == "mean":
        df = df.fillna(df.mean(numeric_only=True))
    else:
        df = df.fillna(method=method)
    print("✅ Missing values filled.")

elif task == "analyze":
    print("\n📊 Summary Statistics:\n")
    print(df.describe(include='all'))

elif task == "generate_rows":
    num = int(input("🔢 How many rows to generate? "))
    sample = df.sample(n=1).to_dict(orient='records')[0]
    new_rows = [sample.copy() for _ in range(num)]
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    print(f"✅ {num} synthetic rows generated.")

elif task == "delete_rows":
    print(f"Current dataset has {len(df)} rows.")
    num = int(input("🔻 How many rows would you like to delete from bottom? "))
    df = df.iloc[:-num] if num < len(df) else df.iloc[0:0]
    print(f"✅ {num} rows deleted.")

elif task == "create_chart":
    print("\n📈 Available columns:\n", df.columns.tolist())
    x = input("Choose X-axis column: ")
    y = input("Choose Y-axis column: ")
    df.plot(x=x, y=y, kind='bar', figsize=(12, 6))
    plt.title(f"{y} vs {x}")
    plt.tight_layout()
    plt.show()

elif task == "make_summary":
    summary_prompt = f"""
You are a data analyst. Summarize this dataset in 5 lines:

{df.head(5).to_string(index=False)}
"""
    summary = llm(summary_prompt, max_new_tokens=300)[0]["generated_text"]
    print("\n📝 Summary:\n", summary)

elif task == "field_meaning":
    print("\n🔎 Semantic Understanding of Columns:")
    embeddings = embedder.encode(df.columns)
    for i, col in enumerate(df.columns):
        print(f"{col}: ➤ Vector: {embeddings[i][:5]}...")  # Only showing first 5 dims

else:
    print("❌ Invalid or unrecognized task.")

# ✅ STEP 9: Save Output
output_name = "updated_dataset.xlsx"
df.to_excel(output_name, index=False)
files.download(output_name)
print(f"\n📥 File saved and downloaded: {output_name}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.5 MB/s eta 0:00:00
